# Navigation

---


### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/VisualBanana.app"`
- **Windows** (x86): `"path/to/VisualBanana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/VisualBanana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/VisualBanana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/VisualBanana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `VisualBanana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="VisualBanana.app")
```

In [2]:
env = UnityEnvironment(file_name="Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Deep Reinforcement Learning!

In order to train your own agent to solve the environment we take few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine!

#### 3.1 Training Loop

The following `train` function encapsulates the training loop. It accepts an `agent` and a dictionary of parameters for the individual components, including replay buffer, policy, and agent.

The dictionary of parameters take the following form:
```python
params = {
    'episodes': 2000,            # number of episodes
    'max_t': 1000,               # maximum length of an episode
    'policy': EpsilonGreedy,
    'policy_params': {
        'eps_start': 1.0,            # starting value for the epsilon-greedy action selection
        'eps_end': 0.01,             # end value for the epsilon-greedy action selection
        'eps_decay': 0.995           # decay rate for the epsilon-greedy action selection
    },
    'agent_params': {
        'state_size': state_size,    # size of the state space
        'action_size': action_size,  # size of the action space
        'seed': 312,                 # seed for the agent
        'alpha': 0.4,                # alpha power value for the prioritized replay buffer sampling
        'gamma': 0.99,               # discount factor
        'tau': 0.001,                # mixing rate soft-update of target parameters
        'lr': 0.0005,                # learning rate
        'update_every': 4,           # update every n-th step
        'network_type': QNetwork,    # network architecture
        'experience_params': {
            'seed': 184,                 # seed for the experience replay buffer
            'buffer_size': 100000,       # size of the replay buffer
            'batch_size': 64             # batch size sampled from the replay buffer
        },
    },
    'maxlen': 100,               # sliding window size of recent scores
    'beta_start': 0.4            # starting value for the beta value of prioritized replay sampling
}
```

In [8]:
from collections import deque

def train(agent, params):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    n_episodes = params.get('episodes', 2000)
    max_t = params.get('max_t', 1000)
    beta_start = params.get('beta_start', 0.4)
    maxlen = params.get('maxlen', 100)
    glie_policy = params.get('policy', None)
    
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=maxlen)  # last N scores
    policy = glie_policy(params.get('policy_params', None))  # initialize epsilon-greedy policy

    beta_schedule = lambda episode: min(1.0, beta_start + episode * (1.0 - beta_start) / n_episodes)

    for i_episode in range(1, n_episodes+1):
        beta = beta_schedule(i_episode)
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, policy)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done, beta)
            score += reward                                # update the score
            state = next_state                             # roll over the state to next time step
            if done:                                       # exit loop if episode finished
                break
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        policy.decay()                    # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

#### 3.2 Replay Buffer

There are two replay buffer implementations, `UniformExperienceBuffer` and `PrioritizedExperienceBuffer`. Both inherit from an abstract base class `ReplayBuffer` that declares the abstract methods:
 - `add()`: add an experience to the buffer of a fixed and confirgurable size
 - `sample()`: sample from the buffer
 - `ready()`: check whether the batch size is smaller than the current buffer size
 
The `UniformExperienceBuffer` implements a uniform sampling strategy that makes no distinction between the individual experiences, while the `PrioritizedExperienceBuffer` implements the strategy set out in the original paper by [Schaul et al.](https://arxiv.org/abs/1511.05952). The experiences are replayed according to the magnitude of the temporal difference error, thus given the agent the opportunity to learn from more meaningful transitions.

The probabilities of sampling from the experience buffer are defined as:

$$P(i) = \frac{p_i^\alpha}{\sum_k p_k^\alpha}$$,

where $p_i$ is the prioritisation of the $i$-th experience and $alpha \in [0; 1]$ is a smoothing parameter. If $alpha$ is equal to zero than this conforms to the uniform sampling strategy and the closer to 1 the stronger the weight towards to the true probabilities.

The priorities, $p_i$, are computed as the magnitude of the TD error plus an additional small positive number for numeric stability in case the error is very close to zero, i.e., $p_i=|\delta_i| + \epsilon$.

We further need to introduce importance-sampling weights, because of the associated bias in the prioritized experience replay:

$$w_i = (\frac{1}{N} * \frac{1}{P(i)})^\beta$$,

where $\beta$ is annealed to 1 towards the end of training. This parameter interacts with the $\alpha$ value in that sampling more aggressively ($\alpha = 1$) is corrected more strongly in the IS weights ($\beta = 1$). The weights, $w_i$, are also scaled by the .

### 4. Experiments

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from rl.agent import DDQN_UER_Agent
from rl.model import QNetwork
from rl.policy import EpsilonGreedy

params = {
    'episodes': 2000,            # number of episodes
    'max_t': 1000,               # maximum length of an episode
    'policy': EpsilonGreedy,
    'policy_params': {
        'eps_start': 1.0,            # starting value for the epsilon-greedy action selection
        'eps_end': 0.01,             # end value for the epsilon-greedy action selection
        'eps_decay': 0.995           # decay rate for the epsilon-greedy action selection
    },
    'agent_params': {
        'state_size': state_size,    # size of the state space
        'action_size': action_size,  # size of the action space
        'seed': 312,                 # seed for the agent
        'alpha': 0.4,                # alpha power value for the prioritized replay buffer sampling
        'gamma': 0.99,               # discount factor
        'tau': 0.001,                # mixing rate soft-update of target parameters
        'lr': 0.0005,                # learning rate
        'update_every': 4,           # update every n-th step
        'network_type': QNetwork,    # network architecture
        'experience_params': {
            'seed': 184,                 # seed for the experience replay buffer
            'buffer_size': 100000,       # size of the replay buffer
            'batch_size': 64             # batch size sampled from the replay buffer
        },
    },
    'maxlen': 100,               # sliding window size of recent scores
    'beta_start': 0.4            # starting value for the beta value of prioritized replay sampling
}
agent = DDQN_UER_Agent(params=params.get('agent_params'))
scores = train(agent=agent, params=params)

df = pd.DataFrame(data={'episode': np.arange(len(scores)), 'score': scores})

# plot the scores
df.plot(kind='line', x='episode',y='score')
df.to_csv('DDQN_UER_QAgent_scores.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from rl.agent import DDQN_UER_Agent
from rl.model import DuelingQNetwork
from rl.policy import EpsilonGreedy

params = {
    'episodes': 2000,            # number of episodes
    'max_t': 1000,               # maximum length of an episode
    'policy': EpsilonGreedy,
    'policy_params': {
        'eps_start': 1.0,            # starting value for the epsilon-greedy action selection
        'eps_end': 0.01,             # end value for the epsilon-greedy action selection
        'eps_decay': 0.995           # decay rate for the epsilon-greedy action selection
    },
    'agent_params': {
        'state_size': state_size,    # size of the state space
        'action_size': action_size,  # size of the action space
        'seed': 312,                 # seed for the agent
        'alpha': 0.4,                # alpha power value for the prioritized replay buffer sampling
        'gamma': 0.99,               # discount factor
        'tau': 0.001,                # mixing rate soft-update of target parameters
        'lr': 0.00001,                # learning rate
        'update_every': 4,           # update every n-th step
        'network_type': DuelingQNetwork,    # network architecture
        'experience_params': {
            'seed': 184,                 # seed for the experience replay buffer
            'buffer_size': 100000,       # size of the replay buffer
            'batch_size': 64             # batch size sampled from the replay buffer
        },
    },
    'maxlen': 100,               # sliding window size of recent scores
    'beta_start': 0.4            # starting value for the beta value of prioritized replay sampling
}

agent = DDQN_UER_Agent(params=params.get('agent_params', None))
scores = train(agent=agent, params=params)

df = pd.DataFrame(data={'episode': np.arange(len(scores)), 'score': scores})

# plot the scores
df.plot(kind='line', x='episode',y='score')
df.to_csv('DDQN_UER_DuelingAgent_scores.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from rl.agent import DDQN_PER_Agent
from rl.model import QNetwork
from rl.policy import EpsilonGreedy

params = {
    'episodes': 2000,            # number of episodes
    'max_t': 1000,               # maximum length of an episode
    'policy': EpsilonGreedy,
    'policy_params': {
        'eps_start': 1.0,            # starting value for the epsilon-greedy action selection
        'eps_end': 0.01,             # end value for the epsilon-greedy action selection
        'eps_decay': 0.995           # decay rate for the epsilon-greedy action selection
    },
    'agent_params': {
        'state_size': state_size,    # size of the state space
        'action_size': action_size,  # size of the action space
        'seed': 312,                 # seed for the agent
        'alpha': 0.4,                # alpha power value for the prioritized replay buffer sampling
        'gamma': 0.99,               # discount factor
        'tau': 0.001,                # mixing rate soft-update of target parameters
        'lr': 0.0005,                # learning rate
        'update_every': 4,           # update every n-th step
        'network_type': QNetwork,    # network architecture
        'experience_params': {
            'seed': 184,                 # seed for the experience replay buffer
            'buffer_size': 100000,       # size of the replay buffer
            'batch_size': 64             # batch size sampled from the replay buffer
        },
    },
    'maxlen': 100,               # sliding window size of recent scores
    'beta_start': 0.4            # starting value for the beta value of prioritized replay sampling
}

agent = DDQN_PER_Agent(params=params.get('agent_params', None))
scores = train(agent=agent, params=params)

df = pd.DataFrame(data={'episode': np.arange(len(scores)), 'score': scores})

# plot the scores
df.plot(kind='line', x='episode',y='score')
df.to_csv('DDQN_PER_QAgent_scores.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from rl.agent import DDQN_PER_Agent
from rl.model import DuelingQNetwork
from rl.policy import EpsilonGreedy

params = {
    'episodes': 2000,            # number of episodes
    'max_t': 1000,               # maximum length of an episode
    'policy': EpsilonGreedy,
    'policy_params': {
        'eps_start': 1.0,            # starting value for the epsilon-greedy action selection
        'eps_end': 0.01,             # end value for the epsilon-greedy action selection
        'eps_decay': 0.995           # decay rate for the epsilon-greedy action selection
    },
    'agent_params': {
        'state_size': state_size,    # size of the state space
        'action_size': action_size,  # size of the action space
        'seed': 312,                 # seed for the agent
        'alpha': 0.4,                # alpha power value for the prioritized replay buffer sampling
        'gamma': 0.99,               # discount factor
        'tau': 0.001,                # mixing rate soft-update of target parameters
        'lr': 0.00001,                # learning rate
        'update_every': 4,           # update every n-th step
        'network_type': DuelingQNetwork,    # network architecture
        'experience_params': {
            'seed': 184,                 # seed for the experience replay buffer
            'buffer_size': 100000,       # size of the replay buffer
            'batch_size': 64             # batch size sampled from the replay buffer
        },
    },
    'maxlen': 100,               # sliding window size of recent scores
    'beta_start': 0.4            # starting value for the beta value of prioritized replay sampling
}

agent = DDQN_PER_Agent(params=params.get('agent_params', None))
scores = train(agent=agent, params=params)

df = pd.DataFrame(data={'episode': np.arange(len(scores)), 'score': scores})

# plot the scores
df.plot(kind='line', x='episode',y='score')
df.to_csv('DDQN_PER_DuelingQAgent_scores.csv', index=False)

Episode 100	Average Score: -0.16
Episode 200	Average Score: 0.292
Episode 300	Average Score: 1.30
Episode 400	Average Score: 2.48
Episode 500	Average Score: 1.94
Episode 600	Average Score: 1.28
Episode 700	Average Score: 0.95
Episode 800	Average Score: 0.82
Episode 900	Average Score: 0.86
Episode 1000	Average Score: 0.83
Episode 1100	Average Score: 0.86
Episode 1200	Average Score: 0.66
Episode 1300	Average Score: 0.64
Episode 1400	Average Score: 0.49
Episode 1500	Average Score: 0.73
Episode 1507	Average Score: 0.79

When finished, you can close the environment.

In [ ]:
env.close()